In [1]:
# set the text search keyword
textSearch = 'covid'
instance = 'tasking-manager-staging-api.hotosm.org'


_query cell hidden for brevity_

In [2]:
import requests
import json
import pandas as pd

def getAllPages(textSearch):
    page = 1
    while True:
        r = requests.get(f'https://{instance}/api/v2/projects/?textSearch={textSearch}&page={page}', 
            headers = {
                'Accept-Language': '*',
                'Content-Type': 'application/json'
            }
        )
        j = json.loads(r.content)
        
        df = pd.DataFrame(json.loads(r.content)['results'])
        yield df
        if j['pagination']['hasNext'] is True:
            page = j['pagination']['nextNum']
        else:
            break

            
result = pd.concat(getAllPages(textSearch), ignore_index=True);
result['country'] = result['country'].apply(lambda x: x[0])  # country string is wrapped in an array

In [3]:
print(list(result.columns))

['activeMappers', 'campaign', 'country', 'dueDate', 'lastUpdated', 'locale', 'mapperLevel', 'name', 'organisationLogo', 'organisationName', 'percentMapped', 'percentValidated', 'priority', 'projectId', 'shortDescription', 'status', 'totalContributors']


## Total project count

In [4]:
print(len(result))

46


# Currently published projects

Drafts and archived project requires PM auth token.

Sorted by country, then project Id

In [5]:
hotosm_colors = {
  "red": "#D73F3F",
  "red-dark": "#6C2020",
  "red-light": "#FFEDED",
  "orange": "#FAA71E",
  "tan": "#F0EFEF",
  "blue-dark": "#2C3038",
  "blue-grey": "#68707F",
  "blue-light": "#929DB3",
  "grey-light": "#E1E0E0",
  "green": "#53AE62"
  }

from matplotlib.colors import LinearSegmentedColormap

alpha = 0.7
colors = [(1,0,0,alpha),(1, 0.6, 0, alpha),(1,1,0,alpha),(0,0.7,0, alpha)]
colors = [hotosm_colors['red'],hotosm_colors['orange'],hotosm_colors['green']]
n_bin = [30, 60, 80, 100]
cm = LinearSegmentedColormap.from_list(
        'RgOrYlGn', colors, N=20)

def highlight_done(val):

    if val > 99.9:
        return 'background-color: "blue"'
    return ''

def highlight_priority(val):
    lookup = {
        'LOW': hotosm_colors['grey-light'],
        'MEDIUM': hotosm_colors['green'],
        'HIGH': hotosm_colors['orange'],
        'URGENT': hotosm_colors['red']
    }
    color = lookup[val]
    return f'background-color: {color}'


relevant = result[['projectId','name','percentMapped', 'percentValidated','priority','status','campaign','country', 'totalContributors']]
pd.set_option('display.max_rows', 500)
sorted_vals = relevant.sort_values(['country','projectId'])
styled = sorted_vals.style\
    .applymap(highlight_done, subset=['percentMapped', 'percentValidated'])\
    .background_gradient(cmap=cm,subset=['percentMapped', 'percentValidated'])\
    .applymap(highlight_priority, subset=['priority'])
    
styled

,projectId,name,percentMapped,percentValidated,priority,status,campaign,country,totalContributors
2,8188,Mapatón COVID-19 Medellín - Comuna 1,66,1,HIGH,PUBLISHED,None,Colombia,263
3,8189,Mapatón COVID-19 Medellín - Comuna 2,77,0,HIGH,PUBLISHED,None,Colombia,194
6,8190,Mapatón COVID-19 Medellín - Comuna 3,15,0,MEDIUM,PUBLISHED,None,Colombia,62
22,8190,Mapatón COVID-19 Medellín - Comuna 3,15,0,MEDIUM,PUBLISHED,None,Colombia,62
8,8191,Mapatón COVID-19 Medellín - Comuna 4,5,0,MEDIUM,PUBLISHED,None,Colombia,7
23,8191,Mapatón COVID-19 Medellín - Comuna 4,5,0,MEDIUM,PUBLISHED,None,Colombia,7
24,8192,Mapatón COVID-19 Medellín - Comuna 5,17,0,MEDIUM,PUBLISHED,None,Colombia,8
25,8193,Mapatón COVID-19 Medellín - Comuna 6,4,0,MEDIUM,PUBLISHED,None,Colombia,3
14,8202,Mapatón COVID-19 Medellín - Comuna 16,40,1,MEDIUM,PUBLISHED,None,Colombia,33
15,8203,Mapatón COVID-19 Medellín - Comuna 15,42,0,MEDIUM,PUBLISHED,None,Colombia,26
